# 1. Feature Engineering
---

In [55]:
import typing
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

---

In [56]:
def ordinal(entry: str) -> int:
    """
    Function that transforms Qualitative/Ordinal values into integers 
    """
    if entry == "Not equipped" or entry == "To be renovated":
        return 1
    elif entry == "Partially equipped" or entry == "Normal":
        return 2
    elif entry == "Fully equipped" or entry == "Excellent":
        return 3
    elif entry == "Super equipped" or entry == "Fully renovated":
        return 4
    elif entry == "New":
        return 5

In [57]:
def empty_as_one(entry: None) -> int:
    """
    Function that replaces floats with integer and empty values with default value "1"
    """
    if pd.isnull(entry):
        return 1
    else:
        return int(entry)

In [58]:
def empty_as_zero(entry: None) -> int:
    """
    Function that replaces object types with integer and empty values with default value "0"
    """
    if pd.isnull(entry):
        return 0
    else:
        return int(float(entry))

---

In [59]:
df = pd.read_csv(
    "/Users/tonyanciaux/Documents/AI Bootcamp - BeCode/project3_immovlan_analysis/utils/data_cleaned.csv",
    sep=";",
)

In [60]:
df.head()

,Unnamed: 0,Locality,Type of property,Subtype of property,Price,Number of bedrooms,Livable surface,Kitchen equipment,Number of bathrooms,Number of toilets,...,Terrace,Surface terrace,Surface bedroom 2,Security door,Access for disabled,Sewer Connection,Garden,Surface garden,Surface bedroom 3,Garage
0,0,1000 Brussels,flat,apartment,333500,2.0,100.0,Super equipped,1,1,...,0,0.0,"10,98298429",0,0,1,0,NaN,"8,916937566",0
1,1,1000 Brussels,flat,apartment,379000,1.0,80.0,Super equipped,1,1,...,0,0.0,"8,786387434",0,0,1,0,NaN,"7,133550053",0
2,2,1000 Brussels,flat,apartment,295000,2.0,80.0,Partially equipped,1,1,...,1,4.0,"8,786387434",0,0,1,0,NaN,"7,133550053",0
3,3,1000 Brussels,flat,loft,635000,2.0,217.0,Super equipped,1,3,...,1,40.0,21,1,1,1,0,NaN,"19,34975452",0
4,4,1000 Brussels,flat,loft,595000,2.0,207.0,Partially equipped,2,2,...,1,10.0,16,0,1,1,0,NaN,"18,45806076",0


---
---

### Dropping rows

In [61]:
#  Merging "Type of property" and "Subtype of property" & renaming columns

df = df.rename(
    columns={
        "Surface bedroom 1": "Surface master bedroom",
        "Subtype of property": "Property type",
    }
)

In [62]:
#  Dropping low-value and too strongly correlated columns 

df = df.drop(
    [
        "Unnamed: 0",
        "Type of property", 
        "Security door",
        "Sewer Connection",
        "Access for disabled",
        "Orientation of the front facade",
        "Furnished",
        "Type of property",
        "Surface bedroom 3",
        "Surface bedroom 2",
        "Surface kitchen",
        "Number of toilets",
        "Floor of appartment",
        "Number of floors",
        "Build Year",
        "Entry phone",
        "Elevator",
        "Number of showers",
        "Terrace",
        "Garden",
    ],
    axis=1,
)

In [63]:
print(df.columns)
print("Number of columns:", len(df.columns))

Index(['Locality', 'Property type', 'Price', 'Number of bedrooms',
       'Livable surface', 'Kitchen equipment', 'Number of bathrooms',
       'Balcony', 'State of the property', 'Surface master bedroom',
       'Surface of living-room', 'Cellar', 'Number of facades',
       'Surface terrace', 'Surface garden', 'Garage'],
      dtype='object')
Number of columns: 16


### Categorical Data
#### Nominal Data

In [64]:
nominal_data = df[["Locality", "Property type"]].copy()

In [65]:
nominal_data["Locality"].value_counts()

1000 Brussels           319
1050 Elsene             314
1030 Schaarbeek         293
8370 Blankenberge       258
9300 Aalst              257
                       ... 
4650 Chaineux             1
7973 Grandglise           1
4632 Cérexhe-Heuseux      1
4630 Ayeneux              1
5360 Hamois               1
Name: Locality, Length: 899, dtype: int64

In [66]:
nominal_data["Property type"].value_counts()

residence         7605
apartment         6574
villa              858
ground-floor       486
duplex             447
mixed-building     356
penthouse          334
studio             236
master-house       129
loft                84
cottage             84
bungalow            79
triplex             36
chalet              25
mansion             14
Name: Property type, dtype: int64

#### Ordinal Data

In [67]:
ordinal_data = df[["Kitchen equipment", "State of the property"]].copy()

In [68]:
ordinal_data["Kitchen equipment"].value_counts()

Partially equipped    10171
Super equipped         3905
Fully equipped         2745
Not equipped            526
Name: Kitchen equipment, dtype: int64

#### Interpretation: 

- Super equipped = 4
- Fully equipped = 3
- Partially equipped = 2
- Not equipped = 1

In [69]:
ordinal_data["Kitchen equipment"] = ordinal_data["Kitchen equipment"].apply(ordinal)

In [70]:
df["Kitchen equipment"] = ordinal_data["Kitchen equipment"]

In [71]:
df["Kitchen equipment"].value_counts()

2    10171
4     3905
3     2745
1      526
Name: Kitchen equipment, dtype: int64

In [72]:
ordinal_data["State of the property"].value_counts()

Normal             7884
New                4550
Excellent          2561
To be renovated    1975
Fully renovated     377
Name: State of the property, dtype: int64

#### Interpretation: 
 - New = 5
 - Fully renovated = 4
 - Excellent = 3
 - Normal = 2
 - To be renovated = 1 

In [73]:
ordinal_data["State of the property"] = ordinal_data["State of the property"].apply(ordinal)

In [74]:
df["State of the property"] = ordinal_data["State of the property"]

In [75]:
df["State of the property"].value_counts()

2    7884
5    4550
3    2561
1    1975
4     377
Name: State of the property, dtype: int64

---
---

### Null value

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17347 entries, 0 to 17346
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Locality                17347 non-null  object 
 1   Property type           17347 non-null  object 
 2   Price                   17347 non-null  int64  
 3   Number of bedrooms      16894 non-null  float64
 4   Livable surface         15419 non-null  float64
 5   Kitchen equipment       17347 non-null  int64  
 6   Number of bathrooms     17347 non-null  int64  
 7   Balcony                 17347 non-null  int64  
 8   State of the property   17347 non-null  int64  
 9   Surface master bedroom  15591 non-null  object 
 10  Surface of living-room  15624 non-null  object 
 11  Cellar                  17347 non-null  int64  
 12  Number of facades       17347 non-null  int64  
 13  Surface terrace         16452 non-null  float64
 14  Surface garden          2700 non-null 

In [77]:
#  Duplicate the column with empty values into a new dataframe as a safety net
null_value = df[
    [
        "Number of bedrooms",
        "Livable surface",
        "Surface master bedroom",
        "Surface of living-room",
        "Surface terrace",
        "Surface garden",
    ]
].copy()

---

In [78]:
df["Number of bedrooms"].value_counts()

2.0      5794
3.0      5671
4.0      2181
1.0      1905
5.0       780
6.0       324
7.0       103
8.0        54
9.0        22
10.0       17
12.0       12
11.0       10
14.0        8
16.0        4
20.0        3
13.0        1
40.0        1
23.0        1
22.0        1
15.0        1
333.0       1
Name: Number of bedrooms, dtype: int64

In [79]:
null_value["Number of bedrooms"] = null_value["Number of bedrooms"].apply(empty_as_one)

In [80]:
df["Number of bedrooms"] = null_value["Number of bedrooms"]

In [81]:
df_filtered = df[df['Number of bedrooms'] >= 40]
df_filtered.head()

,Locality,Property type,Price,Number of bedrooms,Livable surface,Kitchen equipment,Number of bathrooms,Balcony,State of the property,Surface master bedroom,Surface of living-room,Cellar,Number of facades,Surface terrace,Surface garden,Garage
4374,2000 Antwerp,residence,7000000,40,NaN,2,1,0,5,NaN,NaN,0,4,NaN,NaN,0
14552,8650 Houthulst,residence,195000,333,155.0,2,1,0,2,"21,03009192","45,40042113",0,2,0.0,NaN,1


In [82]:
df = df.drop(df_filtered.index)

---

In [83]:
null_value["Surface terrace"].value_counts()

0.0       9824
10.0       378
20.0       365
8.0        339
15.0       317
          ... 
112.0        1
340.0        1
1050.0       1
135.0        1
537.0        1
Name: Surface terrace, Length: 146, dtype: int64

In [84]:
null_value["Surface terrace"] = null_value["Surface terrace"].apply(empty_as_zero)

In [85]:
df["Surface terrace"] = null_value["Surface terrace"]

In [86]:
null_value["Surface garden"].value_counts()

100.0     71
50.0      67
200.0     62
40.0      49
300.0     48
          ..
372.0      1
453.0      1
381.0      1
368.0      1
2940.0     1
Name: Surface garden, Length: 677, dtype: int64

In [87]:
null_value["Surface garden"] = null_value["Surface garden"].apply(empty_as_zero)

In [88]:
df["Surface garden"] = null_value["Surface garden"]

--- 

In [89]:
null_value[["Livable surface", "Surface master bedroom", "Surface of living-room"]]

,Livable surface,Surface master bedroom,Surface of living-room
0,100.0,"13,56780124","29,29059427"
1,80.0,21,25
2,80.0,"10,85424099",28
3,217.0,25,"63,56058958"
4,207.0,16,"60,63153015"
...,...,...,...
17342,99.0,"13,43212323","28,99768833"
17343,NaN,NaN,NaN
17344,630.0,"85,47714781","184,5307439"
17345,196.0,"26,59289043","57,40956478"


Calculating mean surface for living room:

In [90]:
#  Changing NaN values into 0's
null_value["Livable surface"] = null_value["Livable surface"].fillna(0)
null_value["Livable surface"] = null_value["Livable surface"].astype(int)

#  The mean of every value different from 0
nan_filter = null_value["Livable surface"] > 0
mean_surface = int(round(null_value[nan_filter]["Livable surface"].mean(), 0))
print(mean_surface)

158


Replacing 0 with calculated mean:

In [91]:
null_value["Livable surface"] = null_value["Livable surface"].replace(0, mean_surface)

In [92]:
df["Livable surface"] = null_value["Livable surface"]

Calculating mean surface for living room:

In [93]:
null_value["Surface of living-room"] = null_value["Surface of living-room"].fillna(0)
# replacing "," in float numbers with "."
null_value["Surface of living-room"] = null_value["Surface of living-room"].str.replace(",", ".")
null_value["Surface of living-room"] = null_value["Surface of living-room"].apply(
    empty_as_zero
)

nan_livingroom_filter = null_value["Surface of living-room"] > 0
mean_livingroom_surface = int(
    round(null_value[nan_livingroom_filter]["Surface of living-room"].mean(), 0)
)
print(mean_livingroom_surface)

42


Replacing 0's with calculated mean:

In [94]:
null_value["Surface of living-room"] = null_value["Surface of living-room"].replace(
    0, mean_livingroom_surface
)

In [95]:
df["Surface of living-room"] = null_value["Surface of living-room"]

Mean surface for master bedroom:

In [96]:
null_value["Surface master bedroom"] = null_value["Surface master bedroom"].fillna(0)
null_value["Surface master bedroom"] = null_value["Surface master bedroom"].str.replace(
    ",", "."
)
null_value["Surface master bedroom"] = null_value["Surface master bedroom"].apply(
    empty_as_zero
)

nan_bedroom_filter = null_value["Surface master bedroom"] > 0
mean_bedroom_surface = int(
    round(null_value[nan_bedroom_filter]["Surface master bedroom"].mean(), 0)
)
print(mean_bedroom_surface)

20


Replacing 0's with mean:

In [97]:
null_value["Surface master bedroom"] = null_value["Surface master bedroom"].replace(
    0, mean_bedroom_surface
)

In [98]:
df["Surface master bedroom"] = null_value["Surface master bedroom"]

### One Hot Encoding

In [99]:
zipcodes = {
    ("1000", "1299"): "Brussels-Capital Region",
    ("1300", "1499"): "Walloon Brabant Province",
    ("1500", "1999"): "Flemish Brabant Province",
    ("2000", "2999"): "Antwerp Province",
    ("3000", "3499"): "Flemish Brabant Province",
    ("3500", "3999"): "Limburg Province",
    ("4000", "4999"): "Liège Province",
    ("5000", "5999"): "Namur Province",
    ("6000", "6599"): "Hainaut Province",
    ("6600", "6999"): "Luxembourg Province",
    ("7000", "7999"): "Hainaut Province",
    ("8000", "8999"): "West Flanders Province",
    ("9000", "9999"): "East Flanders Province",
}

In [100]:
nominal_data.head()

,Locality,Property type
0,1000 Brussels,apartment
1,1000 Brussels,apartment
2,1000 Brussels,apartment
3,1000 Brussels,loft
4,1000 Brussels,loft


In [101]:
nominal_data["Locality"] = nominal_data["Locality"].str.extract(r"(^\d+)")

In [102]:
for k, v in zipcodes.items():
    nominal_data["Locality"] = np.where(
        nominal_data["Locality"].between(k[0], k[1]), v, nominal_data["Locality"]
    )

In [103]:
nominal_data["Locality"].value_counts()

Antwerp Province            2841
West Flanders Province      2775
Brussels-Capital Region     2627
Hainaut Province            2331
Flemish Brabant Province    1757
East Flanders Province      1694
Liège Province              1185
Limburg Province            1010
Walloon Brabant Province     560
Luxembourg Province          321
Namur Province               246
Name: Locality, dtype: int64

Creating the columns for property types:

In [104]:
prop_types = [
    "residence",
    "apartment",
    "villa",
    "ground-floor",
    "duplex",
    "mixed-building",
    "penthouse",
    "studio",
    "master-house",
    "loft",
    "cottage",
    "bungalow",
    "triplex",
    "chalet",
    "mansion",
]

In [105]:
dummies = pd.get_dummies(nominal_data["Property type"], columns=prop_types)

In [106]:
dummies

,apartment,bungalow,chalet,cottage,duplex,ground-floor,loft,mansion,master-house,mixed-building,penthouse,residence,studio,triplex,villa
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17342,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17343,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17345,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


Merging *dummies* with the original df

In [107]:
df = pd.concat([df, dummies], axis=1)

In [108]:
ohe_localities = pd.get_dummies(nominal_data["Locality"], columns = zipcodes.values())

In [118]:
df = pd.concat([df, ohe_localities], axis=1)

In [119]:
df = df.drop(["Property type", "Locality"], axis=1)

In [121]:
df.head()

,Price,Number of bedrooms,Livable surface,Kitchen equipment,Number of bathrooms,Balcony,State of the property,Surface master bedroom,Surface of living-room,Cellar,...,Brussels-Capital Region,East Flanders Province,Flemish Brabant Province,Hainaut Province,Limburg Province,Liège Province,Luxembourg Province,Namur Province,Walloon Brabant Province,West Flanders Province
0,333500.0,2.0,100.0,4.0,1.0,0.0,2.0,13.0,29.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,379000.0,1.0,80.0,4.0,1.0,0.0,2.0,21.0,25.0,1.0,...,1,0,0,0,0,0,0,0,0,0
2,295000.0,2.0,80.0,2.0,1.0,0.0,2.0,10.0,28.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,635000.0,2.0,217.0,4.0,1.0,0.0,4.0,25.0,63.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,595000.0,2.0,207.0,2.0,2.0,0.0,4.0,16.0,60.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [137]:
df = df.dropna()

In [138]:
df.to_csv("ML_ready_data.csv", index=False)